<a href="https://colab.research.google.com/github/dongjaeseo/AI_Maths/blob/main/courtsbot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchsummary

In [ ]:
import numpy as np
from torchsummary import summary
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, max_length=32):
        self.texts = texts
        self.labels = labels
        self.max_length = max_length
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = int(self.labels[idx])

        # Tokenize the text
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Your dataset
texts = ['i am happy', 'i am sad', 'movie is good', 'hes a bad guy', 'what a good day', 'gloomy day', 'I hate this movie', 'i just love this coffee', 'i dont want to go to class today', 'creepy idea', 'lovely cat', 'great idea']
labels = [1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1]

# Create an instance of your CustomDataset
custom_dataset = CustomDataset(texts, labels)

# Create a DataLoader
batch_size = 2  # You can adjust this based on your needs
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Example usage
for batch in custom_dataloader:
    print(batch)

In [ ]:
# Step 1: Load Pre-trained Transformer Model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
transformer_model = BertForSequenceClassification.from_pretrained(model_name)

# Step 2: Combine Feature Extraction and Text Classification Model
class CombinedModel(nn.Module):
    def __init__(self, transformer_model, num_classes):
        super(CombinedModel, self).__init__()
        self.transformer_model = transformer_model
        self.text_classification_head = nn.Linear(transformer_model.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.transformer_model(input_ids, attention_mask=attention_mask)

        # For BERT models, the output 'last_hidden_state' is now 'logits'
        logits = outputs.logits

        # Perform text classification using the [CLS] token representation
        return logits

# Step 3: Initialize and Combine the Models
num_classes = 2  # Replace with the number of classes in your text classification task
combined_model = CombinedModel(transformer_model, num_classes)

# Step 4: Training
# Set up your optimizer and loss function
optimizer = AdamW(combined_model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
combined_model.to(device)

# Set up your DataLoader and training loop
# Replace this with your actual DataLoader and training loop
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
train_dataloader = custom_dataloader

num_epochs = 10
for epoch in range(num_epochs):
    combined_model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        logits = combined_model(input_ids, attention_mask)
        loss = criterion(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss:.4f}")

# After training, you can save the model if needed
torch.save(combined_model.state_dict(), 'combined_model.pth')

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from transformers import BertTokenizer
from tqdm import tqdm

# Assuming you have a test dataset (test_dataset) and DataLoader
# Replace this with your actual test dataset loading code

# Your dataset
texts = ['I love this movie', 'i just dont like this coffee', 'i cant wait to go to class today', 'best idea', 'ugly dog', 'not bad idea']
# Corrected labels: 0 for negative, 1 for positive
labels = [1, 0, 1, 1, 0, 1]

# Create an instance of your CustomDataset
test_dataset = CustomDataset(texts, labels)

# Create a DataLoader
batch_size = 2
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Set the model to evaluation mode
combined_model.eval()

# Evaluate the model on the test dataset
all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        logits = combined_model(input_ids, attention_mask)

        # Predictions and probabilities
        probs = torch.nn.functional.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=1).cpu().numpy()
        labels = labels.cpu().numpy()

        # Collect predictions, labels, and probabilities
        all_labels.extend(labels)
        all_preds.extend(preds)
        all_probs.extend(probs.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)
classification_rep = classification_report(all_labels, all_preds)

print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

# Print input texts, true labels, predicted labels, and probabilities
for i in range(len(texts)):
    print(f"Text: {texts[i]}, True Label: {labels[i]}, Predicted Label: {all_preds[i]}, Probabilities: {all_probs[i]}")
